In [1]:
# todo: сгенерировать свой трейн,Посмотреть на отклонения с высоким iou и с низким, 
# определить расхождение у разных пользователей для каждого айтема в тесте и трейне и подправить трейн под расхождение из теста
# обучать по батчам для айтемов. транспонировать айтемы и сделать фичи из них
# линейную регрессию проверить, использовать иоу в предсказании
# снизить тест сайз на трейн тест сплите
# вщять полностью икс мин и икс мах из фрема по иоу

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
np.random.seed(0)

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test_data.csv', 'sample_submit_zero.csv', 'train_answers.csv', 'train_data.csv']


In [2]:
def area(box):
    return (box[2] - box[0]) * (box[3] - box[1])


def intersection_over_union(boxes):
    assert(len(boxes) == 8)
    boxA = boxes[:4].values
    boxB = boxes[4:].values
    
    boxAArea = area(boxA)
    boxBArea = area(boxB)
    
    if (boxAArea == 0 or boxBArea == 0):
        return 0
        
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)

    try:
        iou = interArea / float(boxAArea + boxBArea - interArea)
    except ZeroDivisionError:
        iou = 1
    return iou

votes = pd.read_csv("../input/train_data.csv")
answers = pd.read_csv("../input/train_answers.csv")
test = pd.read_csv("../input/test_data.csv")
data = pd.concat([votes, test], axis=0)
X = data[['Xmin','Ymin', 'Xmax', 'Ymax']].values
clustering = DBSCAN(eps=3, min_samples=2).fit(X)
train =  pd.merge(votes, answers, how = 'left', left_on  = ['itemId'], right_on = ['itemId'])
train["iou"] = train[['Xmin','Ymin', 'Xmax', 'Ymax', 'Xmin_true', 'Ymin_true', 'Xmax_true','Ymax_true']].apply(intersection_over_union, axis=1)
train.to_csv("train_iou.csv",index=False)

# # 
# train = train[0:200]
# # 

from shapely.geometry import box
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train[['Xmin','Ymin', 'Xmax', 'Ymax']],test[['Xmin','Ymin', 'Xmax', 'Ymax']])
from sklearn.preprocessing import MinMaxScaler
scaler2 = MinMaxScaler()
scaler2.fit(train[['Xmin','Ymin', 'Xmax', 'Ymax']],test[['Xmin','Ymin', 'Xmax', 'Ymax']])

def make_feature(df):
    df2 = df.copy()
    dfmean = df2.groupby("itemId")[['Xmin','Ymin', 'Xmax', 'Ymax']].mean().reset_index()
    dfmin = df2.groupby("itemId")[['Xmin','Ymin', 'Xmax', 'Ymax']].min().reset_index()
    dfmax = df2.groupby("itemId")[['Xmin','Ymin', 'Xmax', 'Ymax']].max().reset_index()
    dfstd = df2.groupby("itemId")[['Xmin','Ymin', 'Xmax', 'Ymax']].std().reset_index()
    df =  pd.merge(df, dfmean, how = 'left',
                  left_on  = ['itemId'],
                  right_on = ['itemId'], suffixes=("","_mean"))
    df =  pd.merge(df, dfmin, how = 'left',
                  left_on  = ['itemId'],
                  right_on = ['itemId'], suffixes=("","_min"))
    df =  pd.merge(df, dfmax, how = 'left',
                  left_on  = ['itemId'],
                  right_on = ['itemId'], suffixes=("","_max"))
    df =  pd.merge(df, dfstd, how = 'left',
                  left_on  = ['itemId'],
                  right_on = ['itemId'], suffixes=("","_std"))

    dfmean = df2.groupby("userId")[['Xmin','Ymin', 'Xmax', 'Ymax']].mean().reset_index()
    dfmin = df2.groupby("userId")[['Xmin','Ymin', 'Xmax', 'Ymax']].min().reset_index()
    dfmax = df2.groupby("userId")[['Xmin','Ymin', 'Xmax', 'Ymax']].max().reset_index()
    dfstd = df2.groupby("userId")[['Xmin','Ymin', 'Xmax', 'Ymax']].std().reset_index()
    df =  pd.merge(df, dfmean, how = 'left',
                  left_on  = ['userId'],
                  right_on = ['userId'], suffixes=("","_mean_us"))
    df =  pd.merge(df, dfmin, how = 'left',
                  left_on  = ['userId'],
                  right_on = ['userId'], suffixes=("","_min_us"))
    df =  pd.merge(df, dfmax, how = 'left',
                  left_on  = ['userId'],
                  right_on = ['userId'], suffixes=("","_max_us"))
    df =  pd.merge(df, dfstd, how = 'left',
                  left_on  = ['userId'],
                  right_on = ['userId'], suffixes=("","_std_us"))
    
    df['d1'] = df['Xmin_mean']-df['Xmin']
    df['d2'] = df['Ymin_mean']-df['Ymin']
    df['d3'] = df['Xmax_mean']-df['Xmax']
    df['d4'] = df['Ymax_mean']-df['Ymax']
    df['d5'] = df['Xmin_std']-df['Xmin']
    df['d6'] = df['Ymin_std']-df['Ymin']
    df['d7'] = df['Xmax_std']-df['Xmax']
    df['d8'] = df['Ymax_std']-df['Ymax']
    
    df['d9'] = df['Xmin']-df['d1']
    df['d10'] = df['Ymin']-df['d2']
    df['d11'] = df['Xmax']-df['d3']
    df['d12'] = df['Ymax']-df['d4']
    df['d13'] = df['Xmin']-df['d5']
    df['d14'] = df['Ymin']-df['d6']
    df['d15'] = df['Xmax']-df['d7']
    df['d16'] = df['Ymax']-df['d8']
    
    for el in ['Xmin','Ymin', 'Xmax', 'Ymax', 'Xmin_mean','Ymin_mean', 'Xmax_mean', 'Ymax_mean', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'd9', 'd10', 'd11', 'd12', 'd13', 'd14', 'd15', 'd16']:
        df[el+'_log10'] = np.log10(df[el])
    
    df['sum'] = df['Xmin']+df['Ymin']+df['Xmax']+df['Ymax']
    df['mult'] = df['Xmin']*df['Ymin']*df['Xmax']*df['Ymax']/100000000
    df['dif'] = df['Xmin']-df['Ymin']-df['Xmax']-df['Ymax']
    df['dif2'] = df['Xmin']-df['Ymin']+df['Xmax']-df['Ymax']
    df['dif3'] = df['Xmin']+df['Ymin']-df['Xmax']+df['Ymax']
    df['a'] = df['Xmax']-df['Xmin']
    df['b'] = df['Ymax']-df['Ymin']
    df['a_mean'] = df['Xmax_mean']-df['Xmin_mean']
    df['b_mean'] = df['Ymax_mean']-df['Ymin_mean']
    df['a_max'] = df['Xmax_max']-df['Xmin_max']
    df['b_max'] = df['Ymax_max']-df['Ymin_max']
    df['a_min'] = df['Xmax_min']-df['Xmin_min']
    df['b_min'] = df['Ymax_min']-df['Ymin_min']
    df['dd1'] = df['a_mean']/2 - df['a']/2
    df['dd2'] = df['b_mean']/2 - df['b']/2
    df['dd3'] = df['a_max']/2 - df['a']/2
    df['dd4'] = df['b_max']/2 - df['b']/2
    df['dd5'] = df['a_min']/2 - df['a']/2
    df['dd6'] = df['b_min']/2 - df['b']/2
    df['aa'] = df['Xmax']-df['Ymin']
    df['bb'] = df['Ymax']-df['Xmin']
    df['ab'] = df['b']>df['a']
    df['S'] = (df['Xmax']-df['Xmin'])*(df['Ymax']-df['Ymin'])
    df['Sob'] = (df['Xmax_max']-df['Xmin_min'])*(df['Ymax_max']-df['Ymin_min'])
    df['P'] = 2*((df['Xmax']-df['Xmin'])+(df['Ymax']-df['Ymin']))
    df['Pob'] = 2*((df['Xmax_max']-df['Xmin_min'])+(df['Ymax_max']-df['Ymin_min']))
    df['dist_from_0'] = ( (df['a']/2)**2 + (df['b']/2)**2 )**(1/2)
    
    xmin = []
    ymin = []
    xmax = []
    ymax = []
    for index, row in df.iterrows():
        itemid = row.itemId
#         print (itemid)
        e = list(df[df.itemId==itemid][['Xmin','Ymin', 'Xmax', 'Ymax']].values[0])
        rect = box(e[0],e[1],e[2],e[3])
        for index2, row2 in df[df.itemId==itemid].iterrows():
            e = []
            e.append(row2.Xmin)
            e.append(row2.Ymin)
            e.append(row2.Xmax)
            e.append(row2.Ymax)
#             print (e)
            rect = rect.intersection(box(e[0],e[1],e[2],e[3]))
#         print (rect.bounds)
        if rect.bounds ==():
            xmin.append(0)
            ymin.append(0)
            xmax.append(0)
            ymax.append(0)
        else:
            xmin.append(rect.bounds[0])
            ymin.append(rect.bounds[1])
            xmax.append(rect.bounds[2])
            ymax.append(rect.bounds[3])
    
    coords1 = pd.DataFrame(xmin, columns=['Xmin_intersec'])
    df = pd.concat([df, coords1], axis=1)
    coords2 = pd.DataFrame(ymin, columns=['Ymin_intersec'])
    df = pd.concat([df, coords2], axis=1)
    coords3 = pd.DataFrame(xmax, columns=['Xmax_intersec'])
    df = pd.concat([df, coords3], axis=1)
    coords4 = pd.DataFrame(ymax, columns=['Ymax_intersec'])
    df = pd.concat([df, coords4], axis=1)
    
    df['S_intersec'] = (df['Xmax_intersec']-df['Xmin_intersec'])*(df['Ymax_intersec']-df['Ymin_intersec'])
    df['P_intersec'] = 2*((df['Xmax_intersec']-df['Xmin_intersec'])+(df['Ymax_intersec']-df['Ymin_intersec']))
    df['Sob/S_intersec'] = df['Sob']/df['S_intersec']
    df['Pob/P_intersec'] = df['Pob']/df['P_intersec']
    df['S_intersec/Sob'] = df['S_intersec']/df['Sob']
    df['P_intersec/Pob'] = df['P_intersec']/df['Pob']
    df['S_intersec/Pob'] = df['S_intersec']/df['Pob']
    df['P_intersec/Sob'] = df['P_intersec']/df['Sob']
    
    for el in ['S_intersec','P_intersec', 'Sob/S_intersec', 'Pob/P_intersec', 'S_intersec/Sob','P_intersec/Pob', 'S_intersec/Pob', 'P_intersec/Sob', 'dd1', 'dd2', 'dd3', 'dd4', 'dd5', 'dd6']:
        df[el+'_log10'] = np.log10(df[el])
    
    gr = df.groupby('userId')['itemId'].nunique().values
    us = sorted(df.userId.unique())
    gr = pd.DataFrame(gr, columns=['count_user_item'])
    us = pd.DataFrame(us, columns=['userId'])
    gr =  pd.concat([us, gr], axis=1, sort=False)
    df =  pd.merge(df, gr, how = 'left',
                      left_on  = ['userId'],
                      right_on = ['userId'])
    gr = df.groupby('itemId')['userId'].nunique().values
    us = sorted(df.itemId.unique())
    gr = pd.DataFrame(gr, columns=['count_item_user'])
    us = pd.DataFrame(us, columns=['itemId'])
    gr =  pd.concat([us, gr], axis=1, sort=False)
    df =  pd.merge(df, gr, how = 'left',
                      left_on  = ['itemId'],
                      right_on = ['itemId'])
    
    
    scan = clustering.fit_predict(df[['Xmin','Ymin', 'Xmax', 'Ymax']].values)
    scan = pd.DataFrame(scan, columns=['dbscan'])
    df = pd.concat([df, scan], axis=1)
    
    df['dx']=df['Xmin']-df['Ymin']
    df['dy']=df['Xmax']-df['Ymax']
    df['xd']=df['Xmin']-df['Ymax']
    df['yd']=df['Xmax']-df['Ymin']
    df['distancexy']=(df['dx']**2+df['dy']**2)**(1/2)
    df['distanceyx']=(df['xd']**2+df['yd']**2)**(1/2)
    df['distancexy_']=(df['dx']**2+df['yd']**2)**(1/2)
    df['distanceyx_']=(df['xd']**2+df['dy']**2)**(1/2)
    
    def get_cl(s):
        if s>=1900:
            return 1
        if s>=1700:
            return 2
        if s>=1500:
            return 3
        if s>=1300:
            return 4
        if s>=1100:
            return 5
        if s>=900:
            return 6
        if s>=700:
            return 7
        if s>=500:
            return 8
        if s>=300:
            return 9
        if s>=100:
            return 10
        else:
            return 0

    for elem in ['Xmin','Ymin', 'Xmax', 'Ymax']:
    
        df[elem+'_cl'] = df[elem].apply(get_cl)
        
    sc = scaler.transform(df[['Xmin','Ymin', 'Xmax', 'Ymax']])
    sc = pd.DataFrame(sc, columns=['Xmin_sc','Ymin_sc', 'Xmax_sc', 'Ymax_sc'])
    df = pd.concat([df, sc], axis=1)
    sc2 = scaler2.transform(df[['Xmin','Ymin', 'Xmax', 'Ymax']])
    sc2 = pd.DataFrame(sc2, columns=['Xmin_sc_m','Ymin_sc_m', 'Xmax_sc_m', 'Ymax_sc_m'])
    df = pd.concat([df, sc2], axis=1)
#     новые добавленные фичи
#     df['dd1dd2'] = np.sqrt((df['dd1']*df['dd1']) + (df['dd2']*df['dd2']))
#     df['dd2dd1'] = np.sqrt((df['dd1']*df['dd1']) - (df['dd2']*df['dd2']))
#     for el in ['Sob/S_intersec', 'Pob/P_intersec', 'S_intersec/Sob','P_intersec/Pob', 'S_intersec/Pob', 'P_intersec/Sob', 'dd1', 'dd2', 'dd3', 'dd4', 'dd5', 'dd6']:
#         df[el+'_log'] = np.log(df[el])
#     df['a/b'] = df['a']/df['b']
#     df['a-b'] = df['a']-df['b']
#     df['dd1_log2'] = np.log2(df['dd1'])
#     df['dd1_log1p'] = np.log1p(df['dd1'])
#     df['dd1_sin'] = np.sin(df['dd1'])
#     df['dd1_cos'] = np.cos(df['dd1'])
#     df['dd1_tan'] = np.tan(df['dd1'])
#     df['dd1_arcsin'] = np.arcsin(df['dd1'])
#     df['dd1_arccos'] = np.arccos(df['dd1'])
#     df['dd1_arctan'] = np.arctan(df['dd1'])
#     df['dd1_degrees'] = np.degrees(df['dd1'])
#     df['dd1_radians'] = np.radians(df['dd1'])
#     df['dd1_deg2rad'] = np.deg2rad(df['dd1'])
#     df['dd1_rad2deg'] = np.rad2deg(df['dd1'])
#     df['dd1_exp'] = np.exp(df['dd1'])
#     df['dd1_expm1'] = np.expm1(df['dd1'])
#     df['dd1_exp2'] = np.exp2(df['dd1'])
#     df['dd1_sqrt'] = np.sqrt(df['dd1'])
#     df['dd1_cbrt'] = np.cbrt(df['dd1'])
#     df['dd1_square'] = np.square(df['dd1'])
#     df['dd2_log2'] = np.log2(df['dd2'])
#     df['dd2_log1p'] = np.log1p(df['dd2'])
#     df['dd2_sin'] = np.sin(df['dd2'])
#     df['dd2_cos'] = np.cos(df['dd2'])
#     df['dd2_tan'] = np.tan(df['dd2'])
#     df['dd2_arcsin'] = np.arcsin(df['dd2'])
#     df['dd2_arccos'] = np.arccos(df['dd2'])
#     df['dd2_arctan'] = np.arctan(df['dd2'])
#     df['dd2_degrees'] = np.degrees(df['dd2'])
#     df['dd2_radians'] = np.radians(df['dd2'])
#     df['dd2_deg2rad'] = np.deg2rad(df['dd2'])
#     df['dd2_rad2deg'] = np.rad2deg(df['dd2'])
#     df['dd2_exp'] = np.exp(df['dd2'])
#     df['dd2_expm1'] = np.expm1(df['dd2'])
#     df['dd2_exp2'] = np.exp2(df['dd2'])
#     df['dd2_sqrt'] = np.sqrt(df['dd2'])
#     df['dd2_cbrt'] = np.cbrt(df['dd2'])
#     df['dd2_square'] = np.square(df['dd2'])
    
    return df

# df_test = make_feature(test)
# df_train = make_feature(train)

In [3]:
first_bad_us = [969, 415, 1677, 1504, 681, 1601, 314, 1065, 138, 698, 1086, 25, 1377, 426, 145, 706, 247, 482, 132, 537] # 0.5680686 Максимум от этого дропа на минмаксе
# sec_bad_us = [1340, 1324, 11, 207, 931, 197, 1270, 1528, 1378, 1595, 1142, 338, 590, 657, 413, 1191, 1061, 362, 1316, 464, 1514, 952, 1037, 1303, 1239, 1654, 1322]
# th_bad_us = [409, 761, 1006, 1503, 937, 1300, 1573, 30, 191, 1459, 1163, 404, 291, 1732, 335, 496, 786, 843, 465, 261, 318, 311, 1156, 289, 1045, 651, 1466, 407, 525, 854, 848, 68, 567, 1540, 1400, 1068, 287, 1391, 360, 859, 748, 692, 1304, 315, 782, 263, 1476, 27, 842, 1361, 862, 2, 1508, 369, 488, 1047, 448, 252, 1483, 710, 1187, 141, 480, 91, 866, 1055, 379, 283, 909, 349, 1024, 1198, 961, 1332]
df_test_for_cat = test.copy()
print (len(df_test_for_cat))
# first_and_sec = first_bad_us+sec_bad_us
# print (len(first_and_sec))
for el in first_bad_us:
    df = df_test_for_cat[df_test_for_cat.userId==el]
    ind = df.index
    try:
        df_test_for_cat = df_test_for_cat.drop(df_test_for_cat.index[[ind]])
    except IndexError:
        continue

print (len(df_test_for_cat))
print(len(df_test_for_cat.itemId.unique()))
print (df_test_for_cat.head())

df_test_for_cat = make_feature(df_test_for_cat)
print (df_test_for_cat.head())

subm_without_fs_bad = df_test_for_cat.groupby("itemId")[['Xmin_min','Ymin_min', 'Xmax_max', 'Ymax_max']].mean().reset_index()
subm_without_fs_bad.to_csv("subm_without_fs_bad.csv",index=False)
subm_without_fs_bad.head()

3615
3555
630
   userId  itemId  Xmin  Ymin  Xmax  Ymax
0    1581   34804    86   640   242   743
1    1351   34804    85   655   273   766
2     161   34804    85   648   268   748
3     313    5704    32   618   647  1268
4    1097    5704    25   620   602  1242
   userId  itemId  Xmin    ...      Ymin_sc_m  Xmax_sc_m  Ymax_sc_m
0    1581   34804    86    ...       0.402145   0.219865   0.323615
1    1351   34804    85    ...       0.412198   0.249759   0.337026
2     161   34804    85    ...       0.407507   0.244937   0.326531
3     313    5704    32    ...       0.387399   0.610415   0.629738
4    1097    5704    25    ...       0.388740   0.567020   0.614577

[5 rows x 154 columns]


,itemId,Xmin_min,Ymin_min,Xmax_max,Ymax_max
0,18,126,634,764,1132
1,19,27,559,119,729
2,33,21,380,414,652
3,62,35,824,639,1262
4,114,41,570,216,780


In [4]:
from sklearn.metrics import mean_squared_error

df_train = make_feature(train)
df_train = df_train.replace([np.inf, -np.inf], np.nan)
df_train = df_train.fillna(0)
df_test_for_cat = df_test_for_cat.replace([np.inf, -np.inf], np.nan)
df_test_for_cat = df_test_for_cat.fillna(0)

def rmsle(y_true, y_pred):
    return 'RMSLE', np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2))), False
def rae(y_true, y_pred):
    return 'RAE', np.sum(np.abs(y_pred - y_true)) / np.sum(np.abs(np.mean(y_true) - y_true)), False
def errors(y_true, y_pred):
    return 'MSE', mean_squared_error(y_true, y_pred),rmsle(y_true, y_pred),rae(y_true, y_pred)

columns_test = [el for el in df_test_for_cat.columns if el not in ['userId', 'itemId']]
columns_train = [el for el in df_train.columns if el not in ['userId', 'itemId']]

# X_train, X_valid, Y_train, Y_valid = train_test_split(df_train[columns_train],  df_train[['Xmin_true','Ymin_true', 'Xmax_true', 'Ymax_true']], random_state=7, test_size=0.25)
# X_train, X_valid, Y_train, Y_valid = df_train[columns_train][0:710],df_train[columns_train][710:943],  df_train[['Xmin_true','Ymin_true', 'Xmax_true', 'Ymax_true']][0:710],df_train[['Xmin_true','Ymin_true', 'Xmax_true', 'Ymax_true']][710:943] 
X_train, X_valid, Y_train, Y_valid = df_train[columns_train][0:4049],df_train[columns_train][4049:5337],  df_train[['Xmin_true','Ymin_true', 'Xmax_true', 'Ymax_true']][0:4049],df_train[['Xmin_true','Ymin_true', 'Xmax_true', 'Ymax_true']][4049:5337] 
X_train1, X_valid1, Y_train1, Y_valid1 = df_train[columns_train],df_train[columns_train][4049:5337],  df_train[['Xmin_true','Ymin_true', 'Xmax_true', 'Ymax_true']],df_train[['Xmin_true','Ymin_true', 'Xmax_true', 'Ymax_true']][4049:5337] 

X_test = df_test_for_cat[columns_test]

In [5]:
from ludwig.api import LudwigModel # Обновить список фич

model_definition = {'input_features': [
    {'name': 'Xmin', 'type': 'numerical'},
    {'name': 'Ymin', 'type': 'numerical'},
    {'name': 'Xmax', 'type': 'numerical'},
    {'name': 'Ymax', 'type': 'numerical'},
    {'name': 'Xmin_mean', 'type': 'numerical'},
    {'name': 'Ymin_mean', 'type': 'numerical'},
    {'name': 'Xmax_mean', 'type': 'numerical'},
    {'name': 'Ymax_mean', 'type': 'numerical'},
    {'name': 'Xmin_min', 'type': 'numerical'},
    {'name': 'Ymin_min', 'type': 'numerical'},
    {'name': 'Xmax_min', 'type': 'numerical'},
    {'name': 'Ymax_min', 'type': 'numerical'},
    {'name': 'Xmin_max', 'type': 'numerical'},
    {'name': 'Ymin_max', 'type': 'numerical'},
    {'name': 'Xmax_max', 'type': 'numerical'},
    {'name': 'Ymax_max', 'type': 'numerical'},
    {'name': 'Xmin_std', 'type': 'numerical'},
    {'name': 'Ymin_std', 'type': 'numerical'},
    {'name': 'Xmax_std', 'type': 'numerical'},
    {'name': 'Ymax_std', 'type': 'numerical'},
    {'name': 'Xmin_mean_us', 'type': 'numerical'},
    {'name': 'Ymin_mean_us', 'type': 'numerical'},
    {'name': 'Xmax_mean_us', 'type': 'numerical'},
    {'name': 'Ymax_mean_us', 'type': 'numerical'},
    {'name': 'Xmin_min_us', 'type': 'numerical'},
    {'name': 'Ymin_min_us', 'type': 'numerical'},
    {'name': 'Xmax_min_us', 'type': 'numerical'},
    {'name': 'Ymax_min_us', 'type': 'numerical'},
    {'name': 'Xmin_max_us', 'type': 'numerical'},
    {'name': 'Ymin_max_us', 'type': 'numerical'},
    {'name': 'Xmax_max_us', 'type': 'numerical'},
    {'name': 'Ymax_max_us', 'type': 'numerical'},
    {'name': 'Xmin_std_us', 'type': 'numerical'},
    {'name': 'Ymin_std_us', 'type': 'numerical'},
    {'name': 'Xmax_std_us', 'type': 'numerical'},
    {'name': 'Ymax_std_us', 'type': 'numerical'},
    {'name': 'sum', 'type': 'numerical'},
    {'name': 'mult', 'type': 'numerical'},
    {'name': 'dif', 'type': 'numerical'},
    {'name': 'dif2', 'type': 'numerical'},
    {'name': 'dif3', 'type': 'numerical'},
    {'name': 'a', 'type': 'numerical'},
    {'name': 'b', 'type': 'numerical'},
    {'name': 'aa', 'type': 'numerical'},
    {'name': 'bb', 'type': 'numerical'},
    {'name': 'ab', 'type': 'binary'},
    {'name': 'S', 'type': 'numerical'},
    {'name': 'Sob', 'type': 'numerical'},
    {'name': 'P', 'type': 'numerical'},
    {'name': 'Pob', 'type': 'numerical'},
    {'name': 'dist_from_0', 'type': 'numerical'},
    {'name': 'Xmin_intersec', 'type': 'numerical'},
    {'name': 'Ymin_intersec', 'type': 'numerical'},
    {'name': 'Xmax_intersec', 'type': 'numerical'},
    {'name': 'Ymax_intersec', 'type': 'numerical'},
    {'name': 'S_intersec', 'type': 'numerical'},
    {'name': 'P_intersec', 'type': 'numerical'},
    {'name': 'Sob/S_intersec', 'type': 'numerical'},
    {'name': 'Pob/P_intersec', 'type': 'numerical'},
    {'name': 'count_user_item', 'type': 'numerical'},
    {'name': 'count_item_user', 'type': 'numerical'},
    {'name': 'dbscan', 'type': 'category'},
    {'name': 'dx', 'type': 'numerical'},
    {'name': 'dy', 'type': 'numerical'},
    {'name': 'xd', 'type': 'numerical'},
    {'name': 'yd', 'type': 'numerical'},
    {'name': 'distancexy', 'type': 'numerical'},
    {'name': 'distanceyx', 'type': 'numerical'},
    {'name': 'distancexy_', 'type': 'numerical'},
    {'name': 'distanceyx_', 'type': 'numerical'},
    {'name': 'Xmin_cl', 'type': 'category'},
    {'name': 'Ymin_cl', 'type': 'category'},
    {'name': 'Xmax_cl', 'type': 'category'},
    {'name': 'Ymax_cl', 'type': 'category'},
    {'name': 'Xmin_sc', 'type': 'numerical'},
    {'name': 'Ymin_sc', 'type': 'numerical'},
    {'name': 'Xmax_sc', 'type': 'numerical'},
    {'name': 'Ymax_sc', 'type': 'numerical'},
    {'name': 'Xmin_sc_m', 'type': 'numerical'},
    {'name': 'Ymin_sc_m', 'type': 'numerical'},
    {'name': 'Xmax_sc_m', 'type': 'numerical'},
    {'name': 'Ymax_sc_m', 'type': 'numerical'},
    {'name': 'dd1', 'type': 'numerical'},
    {'name': 'dd1_log10', 'type': 'numerical'},
    {'name': 'dd2', 'type': 'numerical'},
    {'name': 'dd2_log10', 'type': 'numerical'},
    {'name': 'P_intersec/Pob_log10', 'type': 'numerical'},
    {'name': 'S_intersec/Sob_log10', 'type': 'numerical'},
    {'name': 'd3_log10', 'type': 'numerical'},
    {'name': 'P_intersec/Pob', 'type': 'numerical'},
    {'name': 'S_intersec/Sob', 'type': 'numerical'},
    {'name': 'd4', 'type': 'numerical'},
    {'name': 'dd6', 'type': 'numerical'},
    {'name': 'dd3_log10', 'type': 'numerical'},
    {'name': 'S_intersec/Pob', 'type': 'numerical'},
    {'name': 'Sob/S_intersec_log10', 'type': 'numerical'},
    {'name': 'S_intersec/Pob_log10', 'type': 'numerical'},
    {'name': 'dd6_log10', 'type': 'numerical'},
    {'name': 'd9', 'type': 'numerical'}
], 
                    'output_features': [
                        {'name': 'Xmin_true', 'type': 'numerical'},
                        {'name': 'Ymin_true', 'type': 'numerical'},
                        {'name': 'Xmax_true', 'type': 'numerical'},
                        {'name': 'Ymax_true', 'type': 'numerical'}
                    ],
                   'training': {'batch_size': 32,'epochs': 1000,'early_stop': 50,'learning_rate': 0.001}
                   }
X = X_valid.copy()
model = LudwigModel(model_definition)
train_stats = model.train(X_train[columns_train])
predictions = model.predict(X_valid[columns_test])
X.index = predictions.index
X = pd.concat([X, predictions], axis=1)
X.to_csv("X_valid_ludw.csv",index=False)

X.head(25)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



,Xmin,Ymin,Xmax,Ymax,Xmin_true,Ymin_true,Xmax_true,Ymax_true,iou,Xmin_mean,Ymin_mean,Xmax_mean,Ymax_mean,Xmin_min,Ymin_min,Xmax_min,Ymax_min,Xmin_max,Ymin_max,Xmax_max,Ymax_max,Xmin_std,Ymin_std,Xmax_std,Ymax_std,Xmin_mean_us,Ymin_mean_us,Xmax_mean_us,Ymax_mean_us,Xmin_min_us,Ymin_min_us,Xmax_min_us,Ymax_min_us,Xmin_max_us,Ymin_max_us,Xmax_max_us,Ymax_max_us,Xmin_std_us,Ymin_std_us,Xmax_std_us,...,P_intersec_log10,Sob/S_intersec_log10,Pob/P_intersec_log10,S_intersec/Sob_log10,P_intersec/Pob_log10,S_intersec/Pob_log10,P_intersec/Sob_log10,dd1_log10,dd2_log10,dd3_log10,dd4_log10,dd5_log10,dd6_log10,count_user_item,count_item_user,dbscan,dx,dy,xd,yd,distancexy,distanceyx,distancexy_,distanceyx_,Xmin_cl,Ymin_cl,Xmax_cl,Ymax_cl,Xmin_sc,Ymin_sc,Xmax_sc,Ymax_sc,Xmin_sc_m,Ymin_sc_m,Xmax_sc_m,Ymax_sc_m,Xmin_true_predictions,Ymin_true_predictions,Xmax_true_predictions,Ymax_true_predictions
0,100,391,552,766,165,331,509,942,0.514592,85.272727,364.272727,527.818182,759.272727,28,337,406,664,159,391,574,805,36.378815,18.482916,47.547488,48.462544,119.400000,511.800000,345.400000,679.800000,38,256,107,385,172,757,552,795,54.440794,194.580832,187.237283,...,3.017033,0.578579,0.290035,-0.578579,-0.290035,1.521792,-2.390405,0.000000,1.000000,0.000000,1.290035,0.000000,0.000000,5,11,-1,-291,-214,-666,161,361.216002,685.183917,332.568790,699.536990,10,9,8,7,-0.179060,-1.019948,0.497868,-0.572526,0.107327,0.235255,0.518804,0.337026,69.930832,343.326630,544.781738,756.130676
1,28,372,539,803,165,331,509,942,0.525459,85.272727,364.272727,527.818182,759.272727,28,337,406,664,159,391,574,805,36.378815,18.482916,47.547488,48.462544,110.666667,636.666667,409.000000,877.000000,28,372,112,653,267,925,576,1175,135.463402,277.258604,257.874000,...,3.017033,0.578579,0.290035,-0.578579,-0.290035,1.521792,-2.390405,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3,11,-1,-344,-264,-775,167,433.626567,792.788749,382.393776,818.731336,0,9,8,7,-0.870199,-1.093195,0.445847,-0.436985,0.033024,0.222520,0.506268,0.358601,66.983337,342.393402,544.165161,758.859619
2,74,377,551,779,165,331,509,942,0.524514,85.272727,364.272727,527.818182,759.272727,28,337,406,664,159,391,574,805,36.378815,18.482916,47.547488,48.462544,109.200000,607.200000,298.200000,786.000000,36,377,92,634,272,775,551,901,94.515607,163.063485,228.255778,...,3.017033,0.578579,0.290035,-0.578579,-0.290035,1.521792,-2.390405,0.000000,0.000000,0.000000,0.778151,0.000000,0.000000,5,11,-1,-303,-228,-705,174,379.200475,726.154942,349.406640,740.951415,0,9,8,7,-0.428638,-1.073920,0.493867,-0.524904,0.080495,0.225871,0.517840,0.344606,72.145622,343.971008,531.962402,746.574463
3,87,375,567,793,165,331,509,942,0.538482,85.272727,364.272727,527.818182,759.272727,28,337,406,664,159,391,574,805,36.378815,18.482916,47.547488,48.462544,102.860759,644.012658,343.569620,888.075949,13,157,31,413,534,1371,896,1602,101.964390,281.178039,224.505307,...,3.017033,0.578579,0.290035,-0.578579,-0.290035,1.521792,-2.390405,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,79,11,-1,-288,-226,-706,192,366.087421,731.641989,346.132922,741.290766,0,9,8,7,-0.303849,-1.081630,0.557892,-0.473618,0.093911,0.224531,0.533269,0.352770,76.027649,356.748596,537.541077,746.690613
4,96,350,406,688,165,331,509,942,0.348848,85.272727,364.272727,527.818182,759.272727,28,337,406,664,159,391,574,805,36.378815,18.482916,47.547488,48.462544,128.500000,666.136364,370.840909,897.977273,16,159,34,526,541,1155,842,1491,113.269487,241.068030,249.188725,...,3.017033,0.578579,0.290035,-0.578579,-0.290035,1.521792,-2.390405,1.821335,1.454845,1.720159,1.579784,1.531479,0.000000,44,11,-1,-254,-282,-592,56,379.526020,594.642750,260.099981,655.734702,0,9,9,8,-0.217457,-1.178007,-0.086368,-0.858261,0.103199,0.207775,0.378014,0.291545,56.686783,364.739990,543.423279,739.798279
5,159,382,498,714,165,331,509,942,0.521058,85.272727,364.272727,527.818182,759.272727,28,337,406,664,159,391,574,805,36.378815,18.482916,47.547488,48.462544,135.000000,473.250000

In [6]:
X["iou"] = X[['Xmin_true_predictions', 'Ymin_true_predictions', 'Xmax_true_predictions','Ymax_true_predictions', 'Xmin_true', 'Ymin_true', 'Xmax_true','Ymax_true']].apply(intersection_over_union, axis=1)
print (': iou :',X.iou.mean())

: iou : 0.5249162829874249


In [7]:
predictions = model.predict(X_test)
pred = pd.concat([df_test_for_cat[['userId', 'itemId']], predictions], axis=1)
pred = pred.groupby("itemId")[['Xmin_true_predictions', 'Ymin_true_predictions', 'Xmax_true_predictions','Ymax_true_predictions']].mean().reset_index()
pred.to_csv("pred_ludw.csv",index=False)
pred.head()

,itemId,Xmin_true_predictions,Ymin_true_predictions,Xmax_true_predictions,Ymax_true_predictions
0,18,141.366058,635.260437,763.105652,1080.484863
1,19,36.019588,558.728760,136.500290,678.938232
2,33,54.930378,384.271881,406.838287,655.822937
3,62,105.375847,857.637695,676.055542,1286.927124
4,114,61.084633,569.929321,230.611420,740.981506


In [8]:
X1 = X_valid1.copy()
model = LudwigModel(model_definition)
train_stats = model.train(X_train1[columns_train])
predictions = model.predict(X_valid1[columns_test])
X1.index = predictions.index
X1 = pd.concat([X1, predictions], axis=1)
X1.head(25)

,Xmin,Ymin,Xmax,Ymax,Xmin_true,Ymin_true,Xmax_true,Ymax_true,iou,Xmin_mean,Ymin_mean,Xmax_mean,Ymax_mean,Xmin_min,Ymin_min,Xmax_min,Ymax_min,Xmin_max,Ymin_max,Xmax_max,Ymax_max,Xmin_std,Ymin_std,Xmax_std,Ymax_std,Xmin_mean_us,Ymin_mean_us,Xmax_mean_us,Ymax_mean_us,Xmin_min_us,Ymin_min_us,Xmax_min_us,Ymax_min_us,Xmin_max_us,Ymin_max_us,Xmax_max_us,Ymax_max_us,Xmin_std_us,Ymin_std_us,Xmax_std_us,...,P_intersec_log10,Sob/S_intersec_log10,Pob/P_intersec_log10,S_intersec/Sob_log10,P_intersec/Pob_log10,S_intersec/Pob_log10,P_intersec/Sob_log10,dd1_log10,dd2_log10,dd3_log10,dd4_log10,dd5_log10,dd6_log10,count_user_item,count_item_user,dbscan,dx,dy,xd,yd,distancexy,distanceyx,distancexy_,distanceyx_,Xmin_cl,Ymin_cl,Xmax_cl,Ymax_cl,Xmin_sc,Ymin_sc,Xmax_sc,Ymax_sc,Xmin_sc_m,Ymin_sc_m,Xmax_sc_m,Ymax_sc_m,Xmin_true_predictions,Ymin_true_predictions,Xmax_true_predictions,Ymax_true_predictions
0,100,391,552,766,165,331,509,942,0.514592,85.272727,364.272727,527.818182,759.272727,28,337,406,664,159,391,574,805,36.378815,18.482916,47.547488,48.462544,119.400000,511.800000,345.400000,679.800000,38,256,107,385,172,757,552,795,54.440794,194.580832,187.237283,...,3.017033,0.578579,0.290035,-0.578579,-0.290035,1.521792,-2.390405,0.000000,1.000000,0.000000,1.290035,0.000000,0.000000,5,11,-1,-291,-214,-666,161,361.216002,685.183917,332.568790,699.536990,10,9,8,7,-0.179060,-1.019948,0.497868,-0.572526,0.107327,0.235255,0.518804,0.337026,29.605799,309.340759,539.775452,858.160828
1,28,372,539,803,165,331,509,942,0.525459,85.272727,364.272727,527.818182,759.272727,28,337,406,664,159,391,574,805,36.378815,18.482916,47.547488,48.462544,110.666667,636.666667,409.000000,877.000000,28,372,112,653,267,925,576,1175,135.463402,277.258604,257.874000,...,3.017033,0.578579,0.290035,-0.578579,-0.290035,1.521792,-2.390405,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3,11,-1,-344,-264,-775,167,433.626567,792.788749,382.393776,818.731336,0,9,8,7,-0.870199,-1.093195,0.445847,-0.436985,0.033024,0.222520,0.506268,0.358601,15.371027,299.462708,542.278198,862.553711
2,74,377,551,779,165,331,509,942,0.524514,85.272727,364.272727,527.818182,759.272727,28,337,406,664,159,391,574,805,36.378815,18.482916,47.547488,48.462544,109.200000,607.200000,298.200000,786.000000,36,377,92,634,272,775,551,901,94.515607,163.063485,228.255778,...,3.017033,0.578579,0.290035,-0.578579,-0.290035,1.521792,-2.390405,0.000000,0.000000,0.000000,0.778151,0.000000,0.000000,5,11,-1,-303,-228,-705,174,379.200475,726.154942,349.406640,740.951415,0,9,8,7,-0.428638,-1.073920,0.493867,-0.524904,0.080495,0.225871,0.517840,0.344606,17.843494,297.816803,534.403870,861.810852
3,87,375,567,793,165,331,509,942,0.538482,85.272727,364.272727,527.818182,759.272727,28,337,406,664,159,391,574,805,36.378815,18.482916,47.547488,48.462544,102.860759,644.012658,343.569620,888.075949,13,157,31,413,534,1371,896,1602,101.964390,281.178039,224.505307,...,3.017033,0.578579,0.290035,-0.578579,-0.290035,1.521792,-2.390405,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,79,11,-1,-288,-226,-706,192,366.087421,731.641989,346.132922,741.290766,0,9,8,7,-0.303849,-1.081630,0.557892,-0.473618,0.093911,0.224531,0.533269,0.352770,24.286192,305.577240,541.669861,846.999084
4,96,350,406,688,165,331,509,942,0.348848,85.272727,364.272727,527.818182,759.272727,28,337,406,664,159,391,574,805,36.378815,18.482916,47.547488,48.462544,128.500000,666.136364,370.840909,897.977273,16,159,34,526,541,1155,842,1491,113.269487,241.068030,249.188725,...,3.017033,0.578579,0.290035,-0.578579,-0.290035,1.521792,-2.390405,1.821335,1.454845,1.720159,1.579784,1.531479,0.000000,44,11,-1,-254,-282,-592,56,379.526020,594.642750,260.099981,655.734702,0,9,9,8,-0.217457,-1.178007,-0.086368,-0.858261,0.103199,0.207775,0.378014,0.291545,1.855775,314.914490,538.863525,825.664612
5,159,382,498,714,165,331,509,942,0.521058,85.272727,364.272727,527.818182,759.272727,28,337,406,664,159,391,574,805,36.378815,18.482916,47.547488,48.462544,135.000000,473.250000,

In [9]:
X1["iou"] = X1[['Xmin_true_predictions', 'Ymin_true_predictions', 'Xmax_true_predictions','Ymax_true_predictions', 'Xmin_true', 'Ymin_true', 'Xmax_true','Ymax_true']].apply(intersection_over_union, axis=1)
print (': iou :',X1.iou.mean())

: iou : 0.5274782805551136


In [10]:
predictions = model.predict(X_test)
pred = pd.concat([df_test_for_cat[['userId', 'itemId']], predictions], axis=1)
pred = pred.groupby("itemId")[['Xmin_true_predictions', 'Ymin_true_predictions', 'Xmax_true_predictions','Ymax_true_predictions']].mean().reset_index()
pred.to_csv("pred_ludw1.csv",index=False)
pred.head()

,itemId,Xmin_true_predictions,Ymin_true_predictions,Xmax_true_predictions,Ymax_true_predictions
0,18,79.263062,594.596313,737.879517,1204.865723
1,19,10.150206,569.593445,134.263489,682.159302
2,33,28.266035,363.366852,404.004639,693.619934
3,62,58.029041,817.050476,601.951538,1332.973022
4,114,31.585474,566.008057,219.681625,746.031006
